# Initialise

In [ ]:
# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd /home/luye/workspace/bgcellmodels/GilliesWillshaw/

# print code version (hash of checked out version)
print("\nCode version info:")
!git log -1 # --format="%H"

In [ ]:
# Import plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white")
import numpy as np
import pandas as pd
%matplotlib notebook

# Bokeh for interactive plots
from bokeh.io import push_notebook, output_notebook, show as bokeh_show
from bokeh.plotting import figure as bokeh_figure
output_notebook()

# Import our analysis modules
%load_ext autoreload
%autoreload 1
%aimport optimize.bpop_analysis_stn
%aimport optimize.bpop_analysis_pop

resp_analysis = optimize.bpop_analysis_stn
pop_analysis = optimize.bpop_analysis_pop

import pprint
pp = pprint.PrettyPrinter(indent=2)

## Load data

In [ ]:
# 100 individuals, 100 generations IBEA dataset
checkpoint_files = [
    ['IBEA_100gen',
     '/home/luye/cloudstore_m/simdata/marasco_folding/optimization_run_20171103_2/opt_checkpoints_cdf893c2.pkl',
     '/home/luye/cloudstore_m/simdata/marasco_folding/optimization_run_20171103_2/opt_checkpoints_cdf893c2_settings.pkl'],
    ['NSGA2_100gen',
     '/home/luye/cloudstore_m/simdata/marasco_folding/optimization_run_20171103_1/opt_checkpoints_3210b868.pkl',
     '/home/luye/cloudstore_m/simdata/marasco_folding/optimization_run_20171103_1/opt_checkpoints_3210b868_settings.pkl']
]

opt_data = pd.DataFrame(checkpoint_files, columns=['name', 'checkpoints_file', 'settings_file'])

In [ ]:
# Choose optimisation to analyse
# opt_data['checkpoints_file'][0] # address by row index
opt_name = 'IBEA_100gen'
idx = opt_data.index[opt_data['name'] == opt_name][0] # addres by value in field

cp_file = opt_data['checkpoints_file'][idx]
settings_file = opt_data['settings_file'][idx]

print("Analysing data from files:\n{}\n{}".format(cp_file, settings_file))

In [ ]:
# Load logs file
import cPickle as pickle

# Old pickling method
# with open(checkpoints_file, 'r') as f:
#     checkpoint = pickle.load(f)
#     # old_param_names = pickle.load(f)

# New pickling method
with open(cp_file, "rb") as f:
    while True:
        try:
            checkpoint = pickle.load(f)
        except EOFError:
            break

# Get variables
hof = checkpoint['halloffame']
log = checkpoint['logbook']
pareto_front = checkpoint['paretofront']

In [ ]:
# Load settings file
with open(settings_file, 'r') as f:
    opt_settings = pickle.load(f)
    
pp.pprint(opt_settings)

# Validation protocol responses

- make a random validation protocol
- set up models and params like in opt notebook
- plot all responses like in l5pc_analysis.py
- calculate scores
- select based on scores
    + write to pickle file: selected params so we can instantiate and run bagged opt

In [ ]:
# Distributed logging
from common import logutils

# BluePyOpt
import bluepyopt.ephys as ephys

# Custom BluePyOpt modules
from optimize.bpop_cellmodels import StnFullModel, StnReducedModel
from optimize.bpop_protocols_stn import BpopProtocolWrapper
import optimize.bpop_analysis_stn as resp_analysis
import optimize.bpop_features_stn as features_stn

# Physiology parameters
from evalmodel.cellpopdata import StnModel
from evalmodel.proto_common import StimProtocol

CLAMP_PLATEAU = StimProtocol.CLAMP_PLATEAU
CLAMP_REBOUND = StimProtocol.CLAMP_REBOUND
MIN_SYN_BURST = StimProtocol.MIN_SYN_BURST
SYN_BACKGROUND_HIGH = StimProtocol.SYN_BACKGROUND_HIGH

# Adjust verbosity of loggers
logutils.setLogLevel('quiet', ['marasco', 'folding', 'redops', 'stn_protos', 'bluepyopt.ephys.parameters'])

## Choose validation protocol

In [ ]:
# Protocols to use for optimisation
validation_proto = SYN_BACKGROUND_HIGH

# Collect al frozen mechanisms and parameters required for protocols to work
proto_wrapper = BpopProtocolWrapper.make(validation_proto)
proto_mechs, proto_params = BpopProtocolWrapper.all_mechs_params([proto_wrapper])

## Run validation protocol

In [ ]:
# Run protocols using full model to get responses
full_model = StnFullModel(
                name		= 'StnGillies',
                mechs		= proto_mechs,
                params		= proto_params)

# Set up simulation
nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)
proto_wrapper.ephys_protocol.record_contained_traces = True

full_responses = proto_wrapper.ephys_protocol.run(
                            cell_model		= full_model, 
                            param_values	= {},
                            sim				= nrnsim,
                            isolate			= True)

# Plot results
resp_analysis.plot_responses(full_responses)

In [ ]:
# Make free parameters (locations etc, not values)
# TODO: open saved notebooks, re-save settings file with full parameters, load them here

# Create reduced model and get parameters
red_model = StnReducedModel(
                name		= 'StnFolded',
                fold_method	= 'marasco',
                num_passes	= 7,
                mechs		= proto_mechs,
                params		= proto_params + opt_params)

ind_responses = []
ind_scores = []
for ind in hof:
    
    ind_responses.append(proto_wrapper.ephys_protocol.run(
                            cell_model		= red_model, 
                            param_values	= {},
                            sim				= nrnsim,
                            isolate			= True)